# Westminster Dog Show Data
## Part 3: merging BIS, groups cleaned df

In [1]:
# housekeeping
import pprint as pp
from tqdm import tqdm
from pathlib import Path

# analysis
import pandas as pd
import numpy as np

In [2]:
#dfs import
BIS_path = Path('./data/BIS_data.csv')
BIS_raw = pd.read_csv(BIS_path)
BIS_df = BIS_raw.copy()

groups_path = Path('./data/groups_data.csv')
groups_raw = pd.read_csv(groups_path)
groups_df = groups_raw.copy()

In [3]:
# val clean up
groups_df['year'] = groups_df['year'].astype(int)
groups_df['year'].value_counts()

#locating index where year = 1
groups_df['year'].min()
groups_df[groups_df['year'] == 1]
idx_1 = 453
#rewrite
groups_df['year'].iloc[idx_1] = 1964
groups_df.iloc[idx_1]

#locating index where year = 202
groups_df['year'].min()
groups_df[groups_df['year'] == 202]
idx_202 = 294
#rewrite
groups_df['year'].iloc[idx_202] = 2023
groups_df.iloc[idx_202]

/var/folders/9m/nqg58g110vb7_lk_087zg21c0000gn/T/ipykernel_44159/3413539797.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groups_df['year'].iloc[idx_1] = 1964
/var/folders/9m/nqg58g110vb7_lk_087zg21c0000gn/T/ipykernel_44159/3413539797.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  groups_df['year'].iloc[idx_202] = 2023


year                                                     2023
breed                          American Staffordshire Terrier
dog               GCHP CH Lbk's Rebel And Proud Party Crasher
owner(s)    Traci Chlan Luisma & Lacey Benitez & Ion Ispas...
group                                                 Terrier
Name: 294, dtype: object

In [4]:
#year slicing
min_year = groups_df['year'].min()

# convert
groups_df['breed'].astype(str)

# find index in corresponding min year
BIS_df[BIS_df['year'] == min_year]
cutoff_idx = 16

#sliced @ 1924; min group year is 1924
BIS_sliced = BIS_df[16:].copy().reset_index(drop=True) #no idx col
BIS_sliced.head()

# creating dataframe to reference in function
ref_df = BIS_sliced[['year', 'breed']].copy()
ref_df['BIS'] = True

In [5]:
# merging
merged_df = pd.merge(groups_df, ref_df, left_on=['year', 'breed'], right_on=['year', 'breed'], how='outer')
merged_df['BIS'] = merged_df['BIS'].fillna(False)

In [6]:
# check
merged_df['BIS'].value_counts() 

# merged function tacked on 5 extra values to df tail
# merged_df.tail(10)

BIS
False    540
True     100
Name: count, dtype: int64

In [7]:
# locate and manually clean
patterns = ['^[Ss]porting',
            '^[Tt]errier',
            '^[Hh]ound',
            '^[Hh]ound',
            '^[Hh]erding'
           ]

years = [1974, 2006, 2008, 2015, 2017]

index_list = []

for i in range(len(years)):
    pattern = patterns[i]
    year = years[i]
    
    group = merged_df.loc[(merged_df['year'] == year) & (merged_df['group']
                                                 .str
                                                 .contains(pattern, regex=True)
                                                )]
    dog_i = group.index[0]
    index_list.append(dog_i)
    merged_df['BIS'].iloc[dog_i] = True # rewrite @ index

/var/folders/9m/nqg58g110vb7_lk_087zg21c0000gn/T/ipykernel_44159/4241956374.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['BIS'].iloc[dog_i] = True # rewrite @ index
/var/folders/9m/nqg58g110vb7_lk_087zg21c0000gn/T/ipykernel_44159/4241956374.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['BIS'].iloc[dog_i] = True # rewrite @ index
/var/folders/9m/nqg58g110vb7_lk_087zg21c0000gn/T/ipykernel_44159/4241956374.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

# DO NOT RUN THE MERGE CLEAN UP CELL BELOW MORE THAN ONCE!!!!! WILL RESULT IN INDEXING ERRORS.

In [8]:

# check for rewrite @ index
# print(index_list)
# for i in index_list:
#     pp.pprint(merged_df.iloc[i])

# remove 5 tail end values - DO NOT RUN MORE THAN ONCE.
merged_df.drop(merged_df.tail(5).index, inplace=True)
merged_df

,year,breed,dog,owner(s),group,BIS
0,2023,English Setter,GCHB CH Winchester's An Apple A Day JH,S Nordstrom & C Hodges & R Barnes & S McGraw &...,Sporting,False
1,2022,English Setter,GCHS CH Ciara N' Honeygait Belle Of The Ball F...,Van Jacobsen & Lee Afdahl & Amanda & Vito Ciar...,Sporting,False
2,2021,German Shorthaired Pointer,GCHS CH Clarity Reach The sky VJK-Myst,V. Nunes-Atkinson & Yvonne Hassler-Deterding &...,Sporting,False
3,2020,Golden Retriever,GCHP CH Hillock's Jack Daniel's RA JH CA RATN ...,"Tammy Tomlinson , Jim Cohen & Robert Samios",Sporting,False
4,2019,Sussex Spaniel,GCh. Kamand’s Full Of Beans @ Erinhill,Karen Ann Toner & Amanda W Toner,Sporting,False
...,...,...,...,...,...,...
630,1987,German Shepherd Dog,Ch. Covy Tucker Hill’s Manhattan,Shirlee Braunstein & Jane A. Firestone,Herding,True
631,1986,German Shepherd Dog,Ch. Covy Tucker Hill’s Manhattan,Shirlee Braunstein & Jane A. Firestone,Herding,False
632,1985,Shetland Sheepdog,Ch. Rhodan’s The Windwalker,Kathleen Schmutz,Herding,False
633,1984,German Shepherd Dog,Ch. Covy Tucker Hill’s Manhattan,Shirlee Braunstein & Jane A. Firestone,Herding,False


In [49]:
# getting values for heat map 
def heat_mapping(BIS):
    if BIS:
        return 2
    else:
        return 1
    
merged_df["won"] = merged_df["BIS"].apply(heat_mapping)
merged_df

,year,breed,dog,owner(s),group,BIS,won
0,2023,English Setter,GCHB CH Winchester's An Apple A Day JH,S Nordstrom & C Hodges & R Barnes & S McGraw &...,Sporting,False,1
1,2022,English Setter,GCHS CH Ciara N' Honeygait Belle Of The Ball F...,Van Jacobsen & Lee Afdahl & Amanda & Vito Ciar...,Sporting,False,1
2,2021,German Shorthaired Pointer,GCHS CH Clarity Reach The sky VJK-Myst,V. Nunes-Atkinson & Yvonne Hassler-Deterding &...,Sporting,False,1
3,2020,Golden Retriever,GCHP CH Hillock's Jack Daniel's RA JH CA RATN ...,"Tammy Tomlinson , Jim Cohen & Robert Samios",Sporting,False,1
4,2019,Sussex Spaniel,GCh. Kamand’s Full Of Beans @ Erinhill,Karen Ann Toner & Amanda W Toner,Sporting,False,1
...,...,...,...,...,...,...,...
630,1987,German Shepherd Dog,Ch. Covy Tucker Hill’s Manhattan,Shirlee Braunstein & Jane A. Firestone,Herding,True,2
631,1986,German Shepherd Dog,Ch. Covy Tucker Hill’s Manhattan,Shirlee Braunstein & Jane A. Firestone,Herding,False,1
632,1985,Shetland Sheepdog,Ch. Rhodan’s The Windwalker,Kathleen Schmutz,Herding,False,1
633,1984,German Shepherd Dog,Ch. Covy Tucker Hill’s Manhattan,Shirlee Braunstein & Jane A. Firestone,Herding,False,1


In [50]:
# uneven groupings for hound + herding 
## --> split, resize consistently for 1924-2023, merge
splice_df = merged_df.copy()

hound_df = splice_df[splice_df['group'] == 'Hound'].reset_index(drop=True)
herding_df = splice_df[splice_df['group'] == 'Herding'].reset_index(drop=True)

In [52]:
def concat_df(df):
    min = df.year.min()
    years = np.flip(np.arange(1924, min))
    length = min - 1924
    
    new_df = pd.DataFrame(years, columns=['year'])
    new_df[['breed', 'dog', 'owner(s)']] = "None"

    if "Herding" in df['group'].values:
        new_df['group'] = 'Herding'

    elif "Hound" in df['group'].values:
        new_df['group'] = 'Hound'
    
    new_df['BIS'] = False
    new_df['won'] = 0
    
    combined = pd.concat([df, new_df]).reset_index(drop=True)
    return combined

In [53]:
combined_hound = concat_df(hound_df)
combined_herding = concat_df(herding_df)

combined_herding

,year,breed,dog,owner(s),group,BIS,won
0,2023,Australian Shepherd,GCHP CH Northbay Xsell That's A Wrap,Bette A Evans,Herding,False,1
1,2022,German Shepherd Dog,GCHG CH Gem-N-I River of Urloved CGC,Richard A Waits,Herding,False,1
2,2021,Old English Sheepdog,GCH CH Bugaboo's Courage of Conviction,Colton Johnson & Heather Johnson & Douglas Joh...,Herding,False,1
3,2020,Shetland Sheepdog,GCHP CH. Syringa- Akadia The Corsair,Tyler Crady & Deborah Sirdofsky & Guy Okada & ...,Herding,False,1
4,2019,Bouvier Des Flandres,GCh. Quiche's Major League,Julianna Garrison & Elaine & Louise Paquette,Herding,False,1
...,...,...,...,...,...,...,...
95,1928,None,None,None,Herding,False,0
96,1927,None,None,None,Herding,False,0
97,1926,None,None,None,Herding,False,0
98,1925,None,None,None,Herding,False,0


In [55]:
# new working merged df with each group 6 x 100.

sporting = splice_df[splice_df['group'] == 'Sporting'].reset_index(drop=True)
working = splice_df[splice_df['group'] == 'Working'].reset_index(drop=True)
terrier = splice_df[splice_df['group'] == 'Terrier'].reset_index(drop=True)
toy = splice_df[splice_df['group'] == 'Toy'].reset_index(drop=True)
non_sporting = splice_df[splice_df['group'] == 'Non-Sporting'].reset_index(drop=True)

frames = [sporting, working, terrier, toy, non_sporting, combined_hound, combined_herding]

# concatenate to matrix
top_dogs = pd.concat(frames)
top_dogs['group'].value_counts()

group
Sporting        100
Working         100
Terrier         100
Toy             100
Non-Sporting    100
Hound           100
Herding         100
Name: count, dtype: int64

In [38]:
final_path = Path('./data/top_dogs.csv')
top_dogs.to_csv(final_path)

# graveyard

In [35]:
# testing cell

In [333]:
grouped_df = merged_df.groupby('group', group_keys=True).apply(lambda x: x)
grouped_df

year                 breed  \
group                                     
Herding 594  2023   Australian Shepherd   
        595  2022   German Shepherd Dog   
        596  2021  Old English Sheepdog   
        597  2020     Shetland Sheepdog   
        598  2019  Bouvier Des Flandres   
...           ...                   ...   
Working 289  1928          Shepherd Dog   
        290  1927        Collie (Rough)   
        291  1926        Collie (Rough)   
        292  1925     Doberman Pinscher   
        293  1924          Shepherd Dog   

                                                  dog  \
group                                                   
Herding 594      GCHP CH Northbay Xsell That's A Wrap   
        595      GCHG CH Gem-N-I River of Urloved CGC   
        596    GCH CH Bugaboo's Courage of Conviction   
        597      GCHP CH. Syringa- Akadia The Corsair   
        598                GCh. Quiche's Major League   
...                                               ...   
Working 289               Ch. Cito Von Der Marktfeste   
        290                     Southport Blue Knight   
        291                   Laund Hero of Bellhaven   
        292  Westphalia Prinsessin Ilisa VD Konigstad   
        293                 Ch. Dolf Von Dusternbrook   

                                                      owner(s)    group    BIS  
group                                                                           
Herding 594                                      Bette A Evans  Herding  False  
        595                                    Richard A Waits  Herding  False  
        596  Colton Johnson & Heather Johnson & Douglas Joh...  Herding  False  
        597  Tyler Crady & Deborah Sirdofsky & Guy Okada & ...  Herding  False  
        598       Julianna Garrison & Elaine & Louise Paquette  Herding  False  
...                                                        ...      ...    ...  
Working 289                                      Giralda Farms  Working  False  
        290                                  Bournbrae Kennels  Working  False  
        291                                   Florence B. Ilch  Working  False  
        292                                 Westphalia Kennels  Working  False  
        293                                    Joselle Kennels  Working  False  

[635 rows x 6 columns]

In [210]:
# for each row in 
## if BIS[year] == groups[year] AND BIS[breed] == groups[breed]
## then it's the same dog that won BIS + group 
## --> write new column BIS_win = TRUE based on meeting condition (fn to test?)

def BIS_winner(row):
    best_year = row['year']
    best_breed = row['breed'].strip()
    # sanity check
    print("Value check:\n")
    print(best_year, "\n")
    print(best_breed, "\n")

    #search group_DF
    print("Checking for ref_year AND ref_breed in groups")
    groups_df['BIS'] = (groups_df['year'] == best_year) & (groups_df['breed'].str.contains(best_breed, case=False))
    print("-----------------------")
    
# iterating over rows
ref_df.apply(lambda x: BIS_winner(x), axis=1)

Value check:

1924 

Sealyham Terrier 

Checking for ref_year AND ref_breed in groups
-----------------------
Value check:

1925 

Pointer 

Checking for ref_year AND ref_breed in groups
-----------------------
Value check:

1926 

Wire Fox Terrier 

Checking for ref_year AND ref_breed in groups
-----------------------
Value check:

1927 

Sealyham Terrier 

Checking for ref_year AND ref_breed in groups
-----------------------
Value check:

1928 

Wire Fox Terrier 

Checking for ref_year AND ref_breed in groups
-----------------------
Value check:

1929 

Collie (Rough) 

Checking for ref_year AND ref_breed in groups
-----------------------
Value check:

1930 

Wire Fox Terrier 

Checking for ref_year AND ref_breed in groups
-----------------------
Value check:

1931 

Wire Fox Terrier 

Checking for ref_year AND ref_breed in groups
-----------------------
Value check:

1932 

Pointer 

Checking for ref_year AND ref_breed in groups
-----------------------
Value check:

1933 

Airedale 

/var/folders/9m/nqg58g110vb7_lk_087zg21c0000gn/T/ipykernel_38231/3732077742.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  groups_df['BIS'] = (groups_df['year'] == best_year) & (groups_df['breed'].str.contains(best_breed, case=False))


0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Length: 100, dtype: object